In [12]:
import pandas as pd
import mysql.connector
from datetime import datetime

df = pd.read_csv("traffic_stops.csv", low_memory=False)

In [13]:
df

,stop_date,stop_time,country_name,driver_gender,driver_age_raw,driver_age,driver_race,violation_raw,violation,search_conducted,search_type,stop_outcome,is_arrested,stop_duration,drugs_related_stop,vehicle_number
0,2020-01-01,0:00:00,Canada,M,59,19,Asian,Drunk Driving,Speeding,True,Vehicle Search,Ticket,True,16-30 Min,True,UP76DY3473
1,2020-01-01,0:01:00,India,M,35,58,Other,Other,Other,False,Vehicle Search,Arrest,True,16-30 Min,True,RJ83PZ4441
2,2020-01-01,0:02:00,USA,M,26,76,Black,Signal Violation,Speeding,False,Frisk,Ticket,True,16-30 Min,True,RJ32OM7264
3,2020-01-01,0:03:00,Canada,M,26,76,Black,Speeding,DUI,True,Frisk,Warning,False,0-15 Min,True,RJ76TI3807
4,2020-01-01,0:04:00,Canada,M,62,75,Other,Speeding,Other,False,Vehicle Search,Arrest,True,16-30 Min,False,WB63BB8305
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65533,2020-02-15,12:13:00,India,F,54,48,Black,Other,Other,False,Vehicle Search,Arrest,True,16-30 Min,False,DL56GW6568
65534,2020-02-15,12:14:00,Canada,F,18,35,Hispanic,Seatbelt,Other,True,Vehicle Search,Ticket,False,16-30 Min,True,TN73EO7098
65535,2020-02-15,12:15:00,USA,M,27,41,Asian,Seatbelt,DUI,True,Frisk,Ticket,True,30+ Min,True,GJ33MX8328
65536,2020-02-15,12:16:00,Canada,F,49,63,Black,Seatbelt,Other,False,NaN,Warning,True,0-15 Min,True,KA24UZ8488


In [14]:
# Data Cleaning
df.dropna(axis=1, how='all', inplace=True)


In [15]:
# Fill NaNs with suitable values
df.fillna({
    'driver_age': df['driver_age'].median(),
    'search_type': 'None',
    'stop_duration': 'Unknown',
    'violation': 'Unknown',
    'stop_outcome': 'Unknown',
}, inplace=True)

# Convert date & time to timestamp
df['timestamp'] = pd.to_datetime(df['stop_date'] + ' ' + df['stop_time'])

In [10]:
df

,stop_date,stop_time,country_name,driver_gender,driver_age_raw,driver_age,driver_race,violation_raw,violation,search_conducted,search_type,stop_outcome,is_arrested,stop_duration,drugs_related_stop,vehicle_number
0,2020-01-01,0:00:00,Canada,M,59,19,Asian,Drunk Driving,Speeding,True,Vehicle Search,Ticket,True,16-30 Min,True,UP76DY3473
1,2020-01-01,0:01:00,India,M,35,58,Other,Other,Other,False,Vehicle Search,Arrest,True,16-30 Min,True,RJ83PZ4441
2,2020-01-01,0:02:00,USA,M,26,76,Black,Signal Violation,Speeding,False,Frisk,Ticket,True,16-30 Min,True,RJ32OM7264
3,2020-01-01,0:03:00,Canada,M,26,76,Black,Speeding,DUI,True,Frisk,Warning,False,0-15 Min,True,RJ76TI3807
4,2020-01-01,0:04:00,Canada,M,62,75,Other,Speeding,Other,False,Vehicle Search,Arrest,True,16-30 Min,False,WB63BB8305
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65533,2020-02-15,12:13:00,India,F,54,48,Black,Other,Other,False,Vehicle Search,Arrest,True,16-30 Min,False,DL56GW6568
65534,2020-02-15,12:14:00,Canada,F,18,35,Hispanic,Seatbelt,Other,True,Vehicle Search,Ticket,False,16-30 Min,True,TN73EO7098
65535,2020-02-15,12:15:00,USA,M,27,41,Asian,Seatbelt,DUI,True,Frisk,Ticket,True,30+ Min,True,GJ33MX8328
65536,2020-02-15,12:16:00,Canada,F,49,63,Black,Seatbelt,Other,False,NaN,Warning,True,0-15 Min,True,KA24UZ8488


In [16]:
!pip install SQLAlchemy PyMySQL

In [17]:
import pymysql

In [23]:
# Connect to MySQL (without specifying a database yet)
conn = mysql.connector.connect(
    host="localhost",      # or IP address / RDS endpoint
    user="root",
    password="Balanandy@22",
)
cursor = conn.cursor()

# Create database
cursor.execute("CREATE DATABASE IF NOT EXISTS securecheck")

In [24]:
cursor.execute("USE securecheck")

In [39]:
# Create table
cursor.execute("DROP TABLE IF EXISTS police_logs")

cursor.execute("""
CREATE TABLE police_logs (
    id INT AUTO_INCREMENT PRIMARY KEY,
    stop_date DATE,
    stop_time TIME,
    country_name VARCHAR(100),
    driver_gender VARCHAR(100),
    driver_age INT,
    driver_race VARCHAR(100),
    violation VARCHAR(100),
    search_conducted BOOLEAN,
    search_type VARCHAR(100),
    stop_outcome VARCHAR(100),
    drugs_related_stop BOOLEAN,
    stop_duration VARCHAR(100),
    vehicle_number VARCHAR(100),
    timestamp DATETIME
)
""")
conn.commit()


In [19]:
df.head()

,stop_date,stop_time,country_name,driver_gender,driver_age_raw,driver_age,driver_race,violation_raw,violation,search_conducted,search_type,stop_outcome,is_arrested,stop_duration,drugs_related_stop,vehicle_number,timestamp
0,2020-01-01,0:00:00,Canada,M,59,19,Asian,Drunk Driving,Speeding,True,Vehicle Search,Ticket,True,16-30 Min,True,UP76DY3473,2020-01-01 00:00:00
1,2020-01-01,0:01:00,India,M,35,58,Other,Other,Other,False,Vehicle Search,Arrest,True,16-30 Min,True,RJ83PZ4441,2020-01-01 00:01:00
2,2020-01-01,0:02:00,USA,M,26,76,Black,Signal Violation,Speeding,False,Frisk,Ticket,True,16-30 Min,True,RJ32OM7264,2020-01-01 00:02:00
3,2020-01-01,0:03:00,Canada,M,26,76,Black,Speeding,DUI,True,Frisk,Warning,False,0-15 Min,True,RJ76TI3807,2020-01-01 00:03:00
4,2020-01-01,0:04:00,Canada,M,62,75,Other,Speeding,Other,False,Vehicle Search,Arrest,True,16-30 Min,False,WB63BB8305,2020-01-01 00:04:00


In [41]:
# Convert stop_date and stop_time to proper formats
df['stop_date'] = pd.to_datetime(df['stop_date'], errors='coerce').dt.strftime('%Y-%m-%d')
df['stop_time'] = pd.to_datetime(df['stop_time'], errors='coerce').dt.strftime('%H:%M:%S')

# Replace NaN with None
df = df.where(pd.notnull(df), None)

records = []

for _, row in df.iterrows():
    records.append((
        row['stop_date'],
        row['stop_time'],
        row['country_name'] if pd.notnull(row['country_name']) else None,
        row['driver_gender'] if pd.notnull(row['driver_gender']) else None,
        int(row['driver_age']) if pd.notnull(row['driver_age']) else None,
        row['driver_race'] if pd.notnull(row['driver_race']) else None,
        row['violation'] if pd.notnull(row['violation']) else None,
        row['search_conducted'] if pd.notnull(row['search_conducted']) else None,
        row['search_type'] if pd.notnull(row['search_type']) else None,
        row['stop_outcome'] if pd.notnull(row['stop_outcome']) else None,
        row['drugs_related_stop'] if pd.notnull(row['drugs_related_stop']) else None,
        row['stop_duration'] if pd.notnull(row['stop_duration']) else None,
        row['vehicle_number'] if pd.notnull(row['vehicle_number']) else f'VEH{row.name}',
        row['timestamp'] if pd.notnull(row['timestamp']) else None
    ))


# Bulk insert
insert_query = """
    INSERT INTO police_logs (
        stop_date, stop_time, country_name, driver_gender, driver_age, driver_race,
        violation, search_conducted, search_type, stop_outcome,
        drugs_related_stop, stop_duration, vehicle_number, timestamp
    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""
cursor.executemany(insert_query, records)
conn.commit()


In [34]:
print(df.columns.tolist())


['stop_date', 'stop_time', 'country_name', 'driver_gender', 'driver_age_raw', 'driver_age', 'driver_race', 'violation_raw', 'violation', 'search_conducted', 'search_type', 'stop_outcome', 'is_arrested', 'stop_duration', 'drugs_related_stop', 'vehicle_number', 'timestamp']


In [33]:
!pip install streamlit